In [1]:
#Dependancies

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [2]:
#Reading csv files

demographicDf = pd.read_csv("resources/demographic.csv")
labsDf = pd.read_csv("resources/labs.csv")
medicationsDf = pd.read_csv("resources/medications.csv", encoding="ISO-8859-1")

In [3]:
#Drops conditions in medicationsDf that appear less than 100 times

medValues = medicationsDf["RXDRSD1"].value_counts(dropna=False)
medicationsDf = medicationsDf[medicationsDf['RXDRSD1'].isin(medValues[medValues >= 100].index)]

In [4]:
#Replaces null values with a "Healthy" string

medicationsDf["RXDRSD1"] = medicationsDf["RXDRSD1"].fillna("Healthy")

In [5]:
#Renames the columns in medicationsDf

medicationsDf = medicationsDf.rename(columns=\
                                    {"RXDUSE": "Medications_Taken_Last_30_Days",\
                                      "RXDDRUG": "Generic_Drug_Name",\
                                      "RXDDRGID": "Generic_Drug_Code", \
                                      "RXQSEEN": "Was_Prescription_Container_Seen_By_Interviewer",\
                                      "RXDDAYS": "Duration_Taken_Medications",\
                                      "RXDRSC1": "ICD_10_CM_Code_1",\
                                      "RXDRSC2": "ICD_10_CM_Code_2",\
                                      "RXDRSC3": "ICD_10_CM_Code_3",\
                                      "RXDRSD1": "ICD_10_CM_Code_1_Description",\
                                      "RXDRSD2": "ICD_10_CM_Code_2_Description",\
                                      "RXDRSD3": "ICD_10_CM_Code_3_Description",\
                                      "RXDCOUNT":"The_Number_Of_Prescription_Medicines_Reported"
                                    })

In [6]:
#Setting up the newLab dataframe

#Isolates the SEQN column into it's own dataframe
labSEQN = labsDf[["SEQN"]]

#Sets null values to 0 and non-null values to 1
#newLab = labsDf.notnull().astype('int')

#Drops the current SEQN column which only has values of 1 currently
newLab = labsDf.drop(columns=["SEQN"])

#Brings the isolated SEQN column back into the dataframe
newLab = newLab.join(labSEQN, how='outer')

#Moves the SEQN column to the far left of our dataframe
columnList = list(newLab.columns.values)
columnList.pop(columnList.index('SEQN'))
newLab = newLab[['SEQN']+columnList]

#Drops new null values
#newLab = newLab.dropna()

#Changes 0 values to 2 values for future division
#for col in columnList:
#    newLab[col] = newLab[col].replace(0, 2, regex=True)

In [7]:
#Setting up the newMed dataframe

#Isolates the three columns we will be working with from the medicationsDf
newMed = medicationsDf[["SEQN", "Generic_Drug_Name", "ICD_10_CM_Code_1_Description"]]

#Renames the third column for readability
newMed = newMed.rename(columns={"ICD_10_CM_Code_1_Description": "Condition"})

#Replaces null perscriptions with a string value of "NONE"
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].fillna("NONE")

#Corrects false data
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].replace("99999", "NONE", regex=True)
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].replace("55555", "NONE", regex=True)
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].replace("77777", "NONE", regex=True)

#Cleans up condition names
newMed["Condition"] = newMed["Condition"].replace("Hypothyroidism, unspecified", "Hypothyroidism", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Major depressive disorder, single episode, unspecified", "Major depressive disorder", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Anxiety disorder, unspecified", "Anxiety disorder", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Sleep disorder, unspecified", "Sleep disorder", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Allergic rhinitis, unspecified", "Allergic rhinitis", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Dorsalgia, unspecified", "Dorsalgia", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Chronic obstructive pulmonary disease, unspecified", "Chronic obstructive pulmonary disease", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Neuralgia and neuritis, unspecified", "Neuralgia and neuritis", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Edema, unspecified", "Edema", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Heart failure, unspecified", "Heart failure", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Cardiac arrhythmia, unspecified", "Cardiac arrhythmia", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Allergy, unspecified", "Allergy", regex=True)

In [8]:
#Setting up the toplabsDf

#Creates a dataframe containing the top 15 used labs
toplabsDf = newLab[["SEQN","LBXRDW","LBXRBCSI","LBXHGB","LBXHCT","LBXMCVSI","LBXMCHSI","LBXPLTSI","LBXMPSI","LBXWBCSI","LBDBANO","LBDEONO","LBDMONO","LBDLYMNO","LBXBAPCT","LBXEOPCT","LBXNEPCT","LBXMOPCT","LBXLYPCT","LBDNENO","LBXHA","LBXHBS"]]

#Renames columns in toplabsDf for readability
toplabsDf = toplabsDf.rename(columns=\
                             {"LBXRDW":"Red cell distribution width (%)",
                             "LBXRBCSI":"Red blood cell count (million cells/uL)",
                             "LBXHGB":"Hemoglobin (g/dL)","LBXHCT":"Hematocrit (%)","LBXMCVSI":"Mean cell volume (fL)",
"LBXMCHSI":"Mean cell hemoglobin (pg)",
"LBXPLTSI":"Platelet count (1000 cells/uL)",
"LBXMPSI":"Mean platelet volume (fL)",
"LBXWBCSI":"White blood cell count (1000 cells/uL)",
"LBDBANO":"Basophils number (1000 cells/uL)",
"LBDEONO":"Eosinophils number (1000 cells/uL)",
"LBDMONO":"Monocyte number (1000 cells/uL)",
"LBDLYMNO":"Lymphocyte number (1000 cells/uL)",
"LBXBAPCT":"Basophils percent (%)",
"LBXEOPCT":"Eosinophils percent (%)",
"LBXNEPCT":"Segmented neutrophils percent (%)",
"LBXMOPCT":"Monocyte percent (%)",
"LBXLYPCT":"Lymphocyte percent (%)",
"LBDNENO":"Segmented neutrophils num (1000 cell/uL)",
"LBXHA":"Hepatitis A antibody",
"LBXHBS":"Hepatitis B Surface Antibody"
                             })

#Replaces values of 0 with values of 2 for future division
columnList = list(toplabsDf.columns.values)
columnList.pop(columnList.index('SEQN'))

for col in columnList:
   toplabsDf[col] = toplabsDf[col].replace(0, 2, regex=True)

In [9]:
#Setting up newDem

#Renames the columns in demographicDf
demographicDf = demographicDf.rename(columns=\
                                    {
"SDDSRVYR":"Data release cycle",
"RIDSTATR":"Interview and examination status of the participant.",
"RIAGENDR":"Gender of the participant.",
"RIDAGEYR":"Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age.",
"RIDAGEMN":"Age in months of the participant at the time of screening. Reported for persons aged 24 months or younger at the time of exam (or screening if not examined).",
"RIDRETH1":"Recode of reported race and Hispanic origin information",
"RIDRETH3":"Recode of reported race and Hispanic origin information, with Non-Hispanic Asian Category",
"RIDEXMON":"Six month time period when the examination was performed - two categories: November 1 through April 30, May 1 through October 31.",
"RIDEXAGM":"Age in months of the participant at the time of examination. Reported for persons aged 19 years or younger at the time of examination.",
"DMQMILIZ":"{Have you/Has SP} ever served on active duty in the U.S. Armed Forces, military Reserves, or National Guard? (Active duty does not include training for the Reserves or National Guard, but does include activation, for service in the U.S. or in a foreign country, in support of military or humanitarian operations.)",
"DMQADFC":"Did {you/SP} ever serve in a foreign country during a time of armed conflict or on a humanitarian or peace-keeping mission? (This would include National Guard or reserve or active duty monitoring or conducting peace keeping operations in Bosnia and Kosovo, in the Sinai between Egypt and Israel, or in response to the 2004 tsunami or Haiti in 2010.)",
"DMDBORN4":"In what country {were you/was SP} born?",
"DMDCITZN":"{Are you/Is SP} a citizen of the United States? [Information about citizenship is being collected by the U.S. Public Health Service to perform health related research. Providing this information is voluntary and is collected under the authority of the Public Health Service Act. There will be no effect on pending immigration or citizenship petitions.]",
"DMDYRSUS":"Length of time the participant has been in the US.",
"DMDEDUC3":"What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?",
"DMDEDUC2":"What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?",
"DMDMARTL":"Marital status",
"RIDEXPRG":"Pregnancy status for females between 20 and 44 years of age at the time of MEC exam.",
"SIALANG":"Language of the Sample Person Interview Instrument",
"SIAPROXY":"Was a Proxy respondent used in conducting the Sample Person (SP) interview?",
"SIAINTRP":"Was an interpreter used to conduct the Sample Person (SP) interview?",
"FIALANG":"Language of the Family Interview Instrument",
"FIAPROXY":"Was a Proxy respondent used in conducting the Family Interview?",
"FIAINTRP":"Was an interpreter used to conduct the Family interview?",
"MIALANG":"Language of the MEC CAPI Interview Instrument",
"MIAPROXY":"Was a Proxy respondent used in conducting the MEC CAPI Interview?",
"MIAINTRP":"Was an interpreter used to conduct the MEC CAPI interview?",
"AIALANGA":"Language of the MEC ACASI Interview Instrument",
"DMDHHSIZ":"Total number of people in the Household",
"DMDFMSIZ":"Total number of people in the Family",
"DMDHHSZA":"Number of children aged 5 years or younger in the household",
"DMDHHSZB":"Number of children aged 6-17 years old in the household",
"DMDHHSZE":"Number of adults aged 60 years or older in the household",
"DMDHRGND":"HH reference person's gender",
"DMDHRAGE":"HH reference person's age in years",
"DMDHRBR4":"HH reference person's country of birth",
"DMDHREDU":"HH reference person's education level",
"DMDHRMAR":"HH reference person's marital status",
"DMDHSEDU":"HH reference person's spouse's education level",
"WTINT2YR":"Full sample 2 year interview weight.",
"WTMEC2YR":"Full sample 2 year MEC exam weight.",
"SDMVPSU":"Masked variance unit pseudo-PSU variable for variance estimation",
"SDMVSTRA":"Masked variance unit pseudo-stratum variable for variance estimation",
"INDHHIN2":"Total household income (reported as a range value in dollars)",
"INDFMIN2":"Total family income (reported as a range value in dollars)",
"INDFMPIR":"A ratio of family income to poverty guidelines."})

# Create a new dataframe with selected columns from the demographicDF 
newDem = demographicDf[["SEQN","Gender of the participant.", "Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age." ,"Marital status" ,"Pregnancy status for females between 20 and 44 years of age at the time of MEC exam." , "Did {you/SP} ever serve in a foreign country during a time of armed conflict or on a humanitarian or peace-keeping mission? (This would include National Guard or reserve or active duty monitoring or conducting peace keeping operations in Bosnia and Kosovo, in the Sinai between Egypt and Israel, or in response to the 2004 tsunami or Haiti in 2010.)" , "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?" , "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?"]]

#Renames the columns in newDem
newDem = newDem.rename(columns=\
                                    {"Gender of the participant.": "Gender",\
                                      "Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age.": "Age",\
                                      "Pregnancy status for females between 20 and 44 years of age at the time of MEC exam.": "Pregnancy_Status", \
                                      "Did {you/SP} ever serve in a foreign country during a time of armed conflict or on a humanitarian or peace-keeping mission? (This would include National Guard or reserve or active duty monitoring or conducting peace keeping operations in Bosnia and Kosovo, in the Sinai between Egypt and Israel, or in response to the 2004 tsunami or Haiti in 2010.)": "Served",\
                                      "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?": "Education1",\
                                      "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?": "Education2"
                                     
                                    })

#Drops columns we don't need, such as education, military status, and pregnancy status
newDem = newDem[["SEQN", "Gender", "Age", "Marital status"]]

In [10]:
#Setting up mainDf

#Combines all three dataframes into a main dataframe
mainDf = newDem.join(newMed.set_index("SEQN"), on='SEQN', how='outer')
mainDf = mainDf.join(toplabsDf.set_index("SEQN"), on="SEQN", how="outer")

#Drops duplicate SEQN entries
mainDf = mainDf.drop_duplicates(subset='SEQN',keep='last')

#Drops any remaining rows containing null values
mainDf = mainDf.dropna()

#Replaces 2 values back to 0 values (oops!)
#columnList = list(mainDf.columns.values)

#for col in columnList:
#    mainDf[col] = mainDf[col].replace(2, 0, regex=True)

In [11]:
#forestDf = mainDf[['Gender', 'Age', 'Marital status', 'Red cell distribution width (%)',
#       'Red blood cell count (million cells/uL)', 'Hemoglobin (g/dL)',
#       'Hematocrit (%)', 'Mean cell volume (fL)', 'Mean cell hemoglobin (pg)',
#       'Platelet count (1000 cells/uL)', 'Mean platelet volume (fL)',
#       'White blood cell count (1000 cells/uL)',
#       'Basophils number (1000 cells/uL)',
#       'Eosinophils number (1000 cells/uL)', 'Monocyte number (1000 cells/uL)',
#       'Lymphocyte number (1000 cells/uL)', 'Basophils percent (%)',
#       'Eosinophils percent (%)', 'Segmented neutrophils percent (%)',
#       'Monocyte percent (%)', 'Lymphocyte percent (%)',
#       'Segmented neutrophils num (1000 cell/uL)', 'Hepatitis A antibody',
#       'Hepatitis B Surface Antibody']]

#conDummies = pd.get_dummies(mainDf['Condition'])
#forestDf = forestDf[["Age", "Gender", "Hepatitis A antibody", "Hepatitis B Surface Antibody".join(conDummies, how='outer')

In [12]:
targetList = ["Age", "Gender", "Hepatitis A antibody", "Hepatitis B Surface Antibody"]
conDummies = pd.get_dummies(mainDf['Condition'])

for category in targetList:

    forestDf = mainDf[[category]].join(conDummies, how='outer')
    target = forestDf[category]

    X_train, X_test, y_train, y_test = train_test_split(forestDf, target, random_state=42)

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    clf.score(X_test, y_test)

    rf = RandomForestClassifier(n_estimators=1000)
    rf = rf.fit(X_train, y_train)
    rf.score(X_test, y_test)

    feature_names = forestDf.columns
    print(f"Target: {category}")
    print("----------------------")
    print(sorted(zip(rf.feature_importances_, feature_names), reverse=True))
    print(" ")

Target: Age
----------------------
[(0.9690147536439122, 'Age'), (0.00366477433978933, 'Healthy'), (0.0016098220184707019, 'Pure hypercholesterolemia'), (0.0013889070318128319, 'Essential (primary) hypertension'), (0.0012325364869825474, 'Hypokalemia'), (0.001222434235106232, 'Major depressive disorder'), (0.0011784518690627207, 'Gastro-esophageal reflux disease'), (0.0011452055500589735, 'Hypothyroidism'), (0.0011363969878219278, 'Type 2 diabetes mellitus'), (0.0011205407966213556, 'Dorsalgia'), (0.0011055976811002954, 'Sleep disorder'), (0.0011041257061198016, 'Anxiety disorder'), (0.0010982536041575111, 'Asthma'), (0.0010930933151209416, 'Functional dyspepsia'), (0.00109115138133108, 'Insomnia'), (0.001021619934463932, 'Allergic rhinitis'), (0.0009703404418328261, 'Migraine'), (0.0009633748549945352, 'Edema'), (0.0009269718927137219, 'Long term (current) use of hormonal contraceptives'), (0.0009255229872121168, 'Prevent blood clots'), (0.0008978564137061561, 'Neuralgia and neuritis'

In [14]:
#Filtering mainDf

Main_Data = mainDf

filtered = Main_Data.loc[Main_Data['Condition'] != 'Essential (primary) hypertension']
filtered2 = filtered.loc[Main_Data['Condition'] != 'Allergy']
filtered3 = filtered2.loc[Main_Data['Condition'] != 'Anxiety disorder']
filtered4 = filtered3.loc[Main_Data['Condition'] != 'Asthma']
filtered5 = filtered4.loc[Main_Data['Condition'] != 'Attention-deficit hyperactivity disorders']
filtered6 = filtered5.loc[Main_Data['Condition'] != 'Insomnia']
filtered7 = filtered6.loc[Main_Data['Condition'] != 'Migraine']
filtered8 = filtered7.loc[Main_Data['Condition'] != 'Sleep disorder']
filtered9 = filtered8.loc[Main_Data['Condition'] != 'Major depressive disorder']
filtered10 = filtered9.loc[Main_Data['Condition'] != 'Cardiac arrhythmia']
filtered11 = filtered10.loc[Main_Data['Condition'] != 'Edema']
filtered12 = filtered11.loc[Main_Data['Condition'] != 'Gastro-esophageal reflux disease']
filtered13 = filtered12.loc[Main_Data['Condition'] != 'Long term (current) use of hormonal contraceptives']
filtered14 = filtered13.loc[Main_Data['Condition'] != 'Allergic rhinitis']
filtered15 = filtered14.loc[Main_Data['Condition'] != 'Enlarged prostate']

mainDf = filtered15

In [18]:
le = preprocessing.LabelEncoder()
mainDf['Condition'] = le.fit_transform(mainDf.Condition.values)

C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
target = mainDf["Condition"]

X_train, X_test, y_train, y_test = train_test_split(forestDf, target, random_state=42)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

rf = RandomForestClassifier(n_estimators=1000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

feature_names = forestDf.columns

sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.17586766691619643, 'Age'),
 (0.057213051174208406, 'Platelet count (1000 cells/uL)'),
 (0.05013476488912538, 'Lymphocyte percent (%)'),
 (0.04931563953031737, 'Red blood cell count (million cells/uL)'),
 (0.048455977183295376, 'Mean cell volume (fL)'),
 (0.048356803168431675, 'Red cell distribution width (%)'),
 (0.04835586148717108, 'Monocyte percent (%)'),
 (0.04834203437504073, 'Hematocrit (%)'),
 (0.0473856992334017, 'Segmented neutrophils percent (%)'),
 (0.04689164824699409, 'Eosinophils percent (%)'),
 (0.04564896378851526, 'Mean platelet volume (fL)'),
 (0.044583306763072275, 'Mean cell hemoglobin (pg)'),
 (0.04443425725784653, 'Hemoglobin (g/dL)'),
 (0.04375660417438688, 'White blood cell count (1000 cells/uL)'),
 (0.041109502878732954, 'Segmented neutrophils num (1000 cell/uL)'),
 (0.04007154169310917, 'Lymphocyte number (1000 cells/uL)'),
 (0.0347837473344322, 'Basophils percent (%)'),
 (0.02688129192551105, 'Monocyte number (1000 cells/uL)'),
 (0.02061990697952068, 'Eos

In [25]:
forestDf = mainDf[['Gender', 'Age', 'Red cell distribution width (%)',
       'Red blood cell count (million cells/uL)', 'Hemoglobin (g/dL)',
       'Hematocrit (%)', 'Mean cell volume (fL)',
       'Mean cell hemoglobin (pg)', 'Platelet count (1000 cells/uL)',
       'Mean platelet volume (fL)',
       'White blood cell count (1000 cells/uL)',
       'Basophils number (1000 cells/uL)',
       'Eosinophils number (1000 cells/uL)',
       'Monocyte number (1000 cells/uL)',
       'Lymphocyte number (1000 cells/uL)', 'Basophils percent (%)',
       'Eosinophils percent (%)', 'Segmented neutrophils percent (%)',
       'Monocyte percent (%)', 'Lymphocyte percent (%)',
       'Segmented neutrophils num (1000 cell/uL)', 'Hepatitis A antibody',
       'Hepatitis B Surface Antibody']]
forestDf

,Gender,Age,Red cell distribution width (%),Red blood cell count (million cells/uL),Hemoglobin (g/dL),Hematocrit (%),Mean cell volume (fL),Mean cell hemoglobin (pg),Platelet count (1000 cells/uL),Mean platelet volume (fL),...,Monocyte number (1000 cells/uL),Lymphocyte number (1000 cells/uL),Basophils percent (%),Eosinophils percent (%),Segmented neutrophils percent (%),Monocyte percent (%),Lymphocyte percent (%),Segmented neutrophils num (1000 cell/uL),Hepatitis A antibody,Hepatitis B Surface Antibody
0,1,69,14.0,5.09,15.2,45.4,89.3,29.9,204.0,9.0,...,0.5,2.0,1.2,3.4,42.3,11.0,42.2,2.0,1.0,1.0
1,1,54,13.4,3.84,11.9,36.7,95.4,31.0,314.0,8.4,...,1.0,3.4,0.6,6.1,58.4,7.6,27.3,7.4,2.0,2.0
10,1,65,13.7,4.35,15.2,43.1,99.1,34.9,187.0,7.7,...,0.6,1.4,1.4,3.2,64.0,9.4,22.0,4.0,2.0,2.0
11,2,26,13.1,4.81,15.1,44.2,91.9,31.3,231.0,9.3,...,0.5,1.3,0.7,0.8,70.1,7.7,20.9,4.2,1.0,1.0
14,1,76,13.1,4.85,14.5,43.4,89.4,30.0,181.0,8.8,...,0.5,1.6,1.0,3.5,56.0,9.3,30.2,3.0,1.0,2.0
17,2,33,14.5,4.59,11.9,35.6,77.4,25.8,210.0,9.6,...,0.3,1.2,0.6,1.4,65.2,7.3,25.6,3.0,2.0,1.0
20,1,32,12.9,5.22,15.5,44.7,85.6,29.7,330.0,7.4,...,0.7,3.2,0.6,3.0,46.7,9.2,40.6,3.7,1.0,2.0
23,2,38,13.7,4.77,13.3,40.8,85.7,27.8,209.0,8.8,...,0.5,1.7,1.7,9.6,49.3,8.4,30.8,2.8,2.0,2.0
24,1,50,12.2,4.73,14.8,42.5,89.9,31.3,172.0,8.1,...,0.3,1.4,0.1,0.3,60.7,7.3,31.6,2.7,1.0,1.0
28,1,28,13.3,5.73,16.0,48.5,84.5,28.0,212.0,8.8,...,0.6,2.1,0.3,1.4,58.3,8.8,31.3,4.0,1.0,1.0


In [28]:
mainDf

,SEQN,Gender,Age,Marital status,Generic_Drug_Name,Condition,Red cell distribution width (%),Red blood cell count (million cells/uL),Hemoglobin (g/dL),Hematocrit (%),...,Monocyte number (1000 cells/uL),Lymphocyte number (1000 cells/uL),Basophils percent (%),Eosinophils percent (%),Segmented neutrophils percent (%),Monocyte percent (%),Lymphocyte percent (%),Segmented neutrophils num (1000 cell/uL),Hepatitis A antibody,Hepatitis B Surface Antibody
0,73557,1,69,4.0,INSULIN,12,14.0,5.09,15.2,45.4,...,0.5,2.0,1.2,3.4,42.3,11.0,42.2,2.0,1.0,1.0
1,73558,1,54,1.0,SIMVASTATIN,11,13.4,3.84,11.9,36.7,...,1.0,3.4,0.6,6.1,58.4,7.6,27.3,7.4,2.0,2.0
10,73567,1,65,2.0,NONE,4,13.7,4.35,15.2,43.1,...,0.6,1.4,1.4,3.2,64.0,9.4,22.0,4.0,2.0,2.0
11,73568,2,26,5.0,NONE,4,13.1,4.81,15.1,44.2,...,0.5,1.3,0.7,0.8,70.1,7.7,20.9,4.2,1.0,1.0
14,73571,1,76,1.0,SIMVASTATIN,10,13.1,4.85,14.5,43.4,...,0.5,1.6,1.0,3.5,56.0,9.3,30.2,3.0,1.0,2.0
17,73574,2,33,1.0,NONE,4,14.5,4.59,11.9,35.6,...,0.3,1.2,0.6,1.4,65.2,7.3,25.6,3.0,2.0,1.0
20,73577,1,32,6.0,NONE,4,12.9,5.22,15.5,44.7,...,0.7,3.2,0.6,3.0,46.7,9.2,40.6,3.7,1.0,2.0
23,73580,2,38,1.0,NONE,4,13.7,4.77,13.3,40.8,...,0.5,1.7,1.7,9.6,49.3,8.4,30.8,2.8,2.0,2.0
24,73581,1,50,1.0,NONE,4,12.2,4.73,14.8,42.5,...,0.3,1.4,0.1,0.3,60.7,7.3,31.6,2.7,1.0,1.0
28,73585,1,28,1.0,NONE,4,13.3,5.73,16.0,48.5,...,0.6,2.1,0.3,1.4,58.3,8.8,31.3,4.0,1.0,1.0
